In [2]:
import numpy as np

from cmath import rect, exp, polar
from math import radians, acos, inf, tan

CASO = "081"

In [3]:
# Fonte de tensão do equivalente de Thevenin do nó da subestação
VthA = 13800 / (3 ** 0.5) + 0j
VthB = VthA * rect(1, radians(-120))
VthC = VthA * rect(1, radians(120))

# Impedância do equivalente de Thevenin do nó da subestação
Zth = (VthA ** 2)
Zth /= (10e8 / 3) * exp(-radians(80) * 1j)

IthA = VthA / Zth
IthB = VthB / Zth
IthC = VthC / Zth

Yth = 1 / Zth

Rmax = 10

exec(open(f"data/CAR{CASO}.py", "r").read())
exec(open(f"data/TOP{CASO}.py", "r").read())
exec(open(f"data/VOL{CASO}.py", "r").read())

cargas    = np.array(cargas)
Emedido   = np.array(Emedido)
topologia = np.array(topologia)

Zcarga = np.array([0 + 0j] * len(cargas))

In [64]:
resultado = np.array([])
resposta  = np.array([])

ultima_posicao = len(topologia) + 2
ultima_posicao *= 3

for caso in range(len(Emedido)):
    E10Ameas = complex(Emedido[caso, 1], Emedido[caso, 2])
    E10Bmeas = complex(Emedido[caso, 1], Emedido[caso, 2])
    E10Cmeas = complex(Emedido[caso, 1], Emedido[caso, 2])

    for i in range(len(cargas)):
        Zcarga[i] = (13800 ** 2)
        Zcarga[i] /= 1000 * complex(cargas[i, 1], -cargas[i, 1] * tan(acos(cargas[i, 2])))
        Zcarga[i] /= 3
        
    # Teste do trecho 10-20
    # 1) Inicialização da função a ser otimizada
    resultadof = np.inf
    for pos_falta in range(1, len(topologia) + 1):
        Q = np.zeros((ultima_posicao, ultima_posicao))
        Q[-3:, -3:] = np.eye(3)
        
        for i in range(1, len(topologia) + 1):
            aux      = 3 * i
            no_pai   = int(topologia[i-1, 0] / 10)
            no_filho = int(topologia[i-1, 1] / 10)
            
            if i == pos_falta:
                Q[aux-3:aux, 3*no_pai - 3:3*no_pai] = np.eye(3)
                Q[aux-3:aux, -3:] = -np.eye(3)
                Q[-6:-3, 3 * no_filho - 3:3*no_filho] = np.eye(3)
                Q[-6:-3, -3:] = -np.eye(3)
            else:
                Q[aux-3:aux, 3 * no_pai - 3:3 * no_pai] = np.eye(3)
                Q[aux-3:aux, 3 * no_filho - 3:3 * no_filho]
                
        YPR = np.zeros((ultima_posicao, ultima_posicao), np.complex)
        for i in range(1, len(topologia) + 1):
            topologia_aux = np.zeros(21)
            
            if caso < 6:
                topologia_aux[[3, 11, 19]] = np.mean(topologia[i-1, (3, 11, 19)])
                topologia_aux[[4, 12, 20]] = np.mean(topologia[i-1, (4, 12, 20)])
                topologia_aux[[5, 7, 9, 13, 15, 17]] = np.mean(topologia[i-1, (5, 7, 9, 13, 15, 17)])
                topologia_aux[[6, 8, 10, 14, 16, 18]] = np.mean(topologia[i-1, (6, 8, 10, 14, 16, 18)])
            else:
                topologia_aux = topologia[i-1, :21]
                
            aux = 3 * i 
            
            dist = topologia[i-1, 2]
            YPR[aux - 3, aux - 3] =  1 / (dist * complex(topologia_aux[3], topologia_aux[4]))
            YPR[aux - 3, aux - 2] = -1 / (dist * complex(topologia_aux[5], topologia_aux[6]))
            YPR[aux - 3, aux - 1] = -1 / (dist * complex(topologia_aux[7], topologia_aux[8]))
            YPR[aux - 2, aux - 3] = YPR[aux - 3, aux - 2]
            YPR[aux - 2, aux - 2] =  1 / (dist * complex(topologia_aux[11], topologia_aux[12]))
            YPR[aux - 2, aux - 1] = -1 / (dist * complex(topologia_aux[13], topologia_aux[14]))
            YPR[aux - 1, aux - 3] = YPR[aux - 3, aux - 1]
            YPR[aux - 1, aux - 2] = YPR[aux - 2, aux - 1]
            YPR[aux - 1, aux - 1] =  1 / (dist * complex(topologia_aux[19], topologia_aux[20]))
        
        mat_cargas = np.zeros((ultima_posicao, ultima_posicao), np.complex)
        mat_cargas[(0, 1, 2), (0, 1, 2)] = Yth
        
        for i in range(2, len(topologia) + 1):
            aux = 3 * i
            mat_cargas[(aux - 3, aux - 2, aux - 1), (aux - 2, aux - 2, aux - 1)] = 1 / Zcarga[i - 2]
            
        funcao = np.inf
        for x in np.arange(1, topologia[pos_falta, 2]):
            for rf in np.arange(0.1, Rmax + 0.05, 0.1):
                topologia_aux = np.zeros(21)
            
                if caso < 6:
                    topologia_aux[[3, 11, 19]] = np.mean(topologia[pos_falta-1, (3, 11, 19)])
                    topologia_aux[[4, 12, 20]] = np.mean(topologia[pos_falta-1, (4, 12, 20)])
                    topologia_aux[[5, 7, 9, 13, 15, 17]] = np.mean(topologia[pos_falta-1, (5, 7, 9, 13, 15, 17)])
                    topologia_aux[[6, 8, 10, 14, 16, 18]] = np.mean(topologia[pos_falta-1, (6, 8, 10, 14, 16, 18)])
                else:
                    topologia_aux = topologia[pos_falta-1, :21]
                    
                aux = 3 * pos_falta 
                YPR[aux - 3, aux - 3] =  1 / (x * complex(topologia_aux[3], topologia_aux[4]))
                YPR[aux - 3, aux - 2] = -1 / (x * complex(topologia_aux[5], topologia_aux[6]))
                YPR[aux - 3, aux - 1] = -1 / (x * complex(topologia_aux[7], topologia_aux[8]))
                YPR[aux - 2, aux - 3] = YPR[aux - 3, aux - 2]
                YPR[aux - 2, aux - 2] =  1 / (x * complex(topologia_aux[11], topologia_aux[12]))
                YPR[aux - 2, aux - 1] = -1 / (x * complex(topologia_aux[13], topologia_aux[14]))
                YPR[aux - 1, aux - 3] = YPR[aux - 3, aux - 1]
                YPR[aux - 1, aux - 2] = YPR[aux - 2, aux - 1]
                YPR[aux - 1, aux - 1] =  1 / (x * complex(topologia_aux[19], topologia_aux[20]))
                
                aux = ultima_posicao - 3
#                 dist = topologia[pos_falta-1, 2]                
                dist = topologia_aux[2]
                
                YPR[aux - 3, aux - 3] =  1 / ((dist - x) * complex(topologia_aux[3], topologia_aux[4]))
                YPR[aux - 3, aux - 2] = -1 / ((dist - x) * complex(topologia_aux[5], topologia_aux[6]))
                YPR[aux - 3, aux - 1] = -1 / ((dist - x) * complex(topologia_aux[7], topologia_aux[8]))
                YPR[aux - 2, aux - 3] = YPR[aux - 3, aux - 2]
                YPR[aux - 2, aux - 2] =  1 / ((dist - x) * complex(topologia_aux[11], topologia_aux[12]))
                YPR[aux - 2, aux - 1] = -1 / ((dist - x) * complex(topologia_aux[13], topologia_aux[14]))
                YPR[aux - 1, aux - 3] = YPR[aux - 3, aux - 1]
                YPR[aux - 1, aux - 2] = YPR[aux - 2, aux - 1]
                YPR[aux - 1, aux - 1] =  1 / ((dist - x) * complex(topologia_aux[19], topologia_aux[20]))
                
                YPR[(-3, -2, -1), (-3, -2, -1)] = (3 ** 0.5) / rf
                
                YNOS = Q.T @ YPR @ Q
                YNOS = YNOS + mat_cargas
                
                matI = np.zeros(ultima_posicao, np.complex)
                matI[[0, 1, 2]] = [IthA, IthB, IthC]
                
#                 E = np.linalg.solve(YNOS, matI)
#                 E10Acalc, E10Bcalc, E10Ccalc = E[[0, 1, 2]]
                
#                 funcao_old = (abs(E10Ameas - E10Acalc) / abs(E10Ameas)) + (abs(E10Bmeas - E10Bcalc) / abs(E10Bmeas)) + (abs(E10Cmeas - E10Ccalc) / abs(E10Cmeas))
#                 if funcao_old < funcao:
#                     xcalc = x
#                     rcalc = rf
#                     funcao = funcao_old
        print(f"{caso:03d},{topologia[pos_falta-1, 0]:03d},{topologia[pos_falta-1, 1]:03d},{xcalc:.5f},{rcalc:.5f},{funcao:.5f}")


ValueError: Unknown format code 'd' for object of type 'float'

In [66]:
splinalg.inv(YNOS)

LinAlgError: singular matrix

In [58]:
from scipy.io import savemat
import scipy.linalg as splinalg